# web interaction ds4 mgrs lab

In [2]:
import os, sys
import pprint
import subprocess
import sys
import string
import time
from pathlib import Path
#from colorama import init, Fore, Back, Style
import colorama as cl
import pandas as pd
import splits_aggregator as sa
import extract_rules as er
import metric_graphics as mg
import pq 
from bokeh.plotting import figure, show
from bokeh.layouts import row
from bokeh.io import output_notebook
#from interact import main

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
__author__ = 'John Mark Agosta john-mark.agosta@microsoft.com'

### config constants 
VERBOSE = False
RULES_PER_SAMPLE = 1
RULE_PAIRS =100
ROOT_DIR = Path('C:/Users/joagosta/OneDrive - Microsoft/data/20news')
INTERACTIVE = True
RUN_EX = True
PR = True
PLOT_IT = False

In [5]:
def main(root_dir, run_interactive):

    print('PID:', os.getpid())
    # Test split pattern extraction
    # input_pattern= glob_pattern + '*.csv'
    rules_dir = root_dir / "rules"
    rule_gen = er.RuleExtraction()
    cs = sa.BinaryComparisons(root_dir)
    if RUN_EX:
        rule_gen.extract_rules()
        user_rules = rule_gen.cycle_rules()
    else:
        # pair_df = cs.random_pairs(INTERACTIVE_RULE_PAIRS) # 
        # Save pairs to shared directory 
        # cs.embed_in_excel(pair_df)
        # Seed the model with a few auto-generated splits
        seed_df = cs.random_pairs(RULE_PAIRS)
        user_rules = cs.simulate_splits(seed_df)  # Creates simulated rules. 
    # While running interactively load new excel splits and add them to the rules
    # Load the test set
    test_df = pq.reload_parquet(root_dir / 'test_clean') 
    cl.init()
    summary_df = pd.DataFrame(columns = ['Train_acc','Train_pr', 'Train_rcl', 'Test_acc', 'Test_pr', 'Test_rcl'])
    #user_rule_cnt = -1
    while INTERACTIVE:
        # Check if the rules directory has any new files, then continue. 
        rule_gen.extract_rules()
        new_rules = rule_gen.cycle_rules()
        new_rule_cnt = len(new_rules)
        #new_rule_cnt = 0
        if new_rule_cnt > 0: # user_rule_cnt:
            #users = sa.CollectSplits(rules_dir, SS_SUFFIX)
            user_rules.extend(new_rules)  # combine lists
            if VERBOSE: pprint.pprint(new_rules)
            # Evaluate the ruleset and display. 
            learner = sa.SplitClassifier(user_rules)
            # learner.order_by_hits(cs.full_df)  # Don't need to tune the rules 
            summary_train = learner.compute_confusion(cs.full_df)
            # Do the same for the test data
            summary_test = learner.compute_confusion(test_df)
            summary_train.extend(summary_test)
            # Collect summary
            next_episode = pd.DataFrame([summary_train])
            next_episode.columns = ['Train_acc','Train_pr', 'Train_rcl', 'Test_acc', 'Test_pr', 'Test_rcl']
            summary_df = summary_df.append(next_episode, ignore_index=True)
            summary_df.to_csv(Path(root_dir)/'summary.csv', header=True, index=False)
            if PR:
                print(cl.Fore.YELLOW + cl.Style.BRIGHT + 'Summary') 
                print(summary_df)
                print(cl.Fore.RESET)
            #user_rule_cnt = new_rule_cnt
            if PLOT_IT:
                cm = pd.read_csv(ROOT_DIR / 'cm.csv', header = 0, index_col=0)
                p = mg.matrix_heatmap(cm)
                sum = pd.read_csv(ROOT_DIR / 'summary.csv', header = 0, index_col=0)
                s = mg.summary_accuracy(sum)
            # show(row(p, s))
            time.sleep(30.0)
        else:
            time.sleep(30.0)
            #print('SLEEP')

    return 0

In [7]:
main(ROOT_DIR, INTERACTIVE)

PID: 2044


Reloaded C:\Users\joagosta\OneDrive - Microsoft\data\20news\train_clean\full_df.parquet


User submitted rules:  567
Selected 10


Reloaded C:\Users\joagosta\OneDrive - Microsoft\data\20news\test_clean\full_df.parquet


User submitted rules:  567
Selected 20
Accuracy on matches = 102  /  352  =  0.2897727272727273
Accuracy on matches = 65  /  243  =  0.2674897119341564
Summary
   Train_acc  Train_pr  Train_rcl  Test_acc   Test_pr  Test_rcl
0   0.289773  0.014489   0.014489   0.26749  0.013374  0.014381

User submitted rules:  586
Selected 30
Accuracy on matches = 112  /  615  =  0.1821138211382114
Accuracy on matches = 80  /  416  =  0.19230769230769232
Summary
   Train_acc  Train_pr  Train_rcl  Test_acc   Test_pr  Test_rcl
0   0.289773  0.014489   0.014489  0.267490  0.013374  0.014381
1   0.182114  0.009106   0.015909  0.192308  0.009615  0.017699

User submitted rules:  592
Selected 40
Accuracy on matches = 122  /  639  =  0.19092331768388107
Accuracy on matches = 83  /  427  =  0.19437939110070257
Summary
   Train_acc  Train_pr  Train_rcl  Test_acc   Test_pr  Test_rcl
0   0.289773  0.014489   0.014489  0.267490  0.013374  0.014381
1   0.182114  0.009106   0.015909  0.192308  0.009615  0.017699
2  

12   0.197706  0.040149   0.061943  0.192933  0.038095  0.061795

User submitted rules:  732
Selected 150
Accuracy on matches = 527  /  2674  =  0.19708302169035152
Accuracy on matches = 352  /  1842  =  0.19109663409337677
Summary
    Train_acc  Train_pr  Train_rcl  Test_acc   Test_pr  Test_rcl
0    0.289773  0.014489   0.014489  0.267490  0.013374  0.014381
1    0.182114  0.009106   0.015909  0.192308  0.009615  0.017699
2    0.190923  0.029939   0.017069  0.194379  0.023252  0.018231
3    0.173951  0.017329   0.026002  0.173750  0.017298  0.028160
4    0.177546  0.017720   0.026582  0.177419  0.017692  0.028869
5    0.199595  0.044295   0.037066  0.208333  0.044050  0.041003
6    0.238527  0.045918   0.048115  0.235447  0.042908  0.049949
7    0.238590  0.042318   0.050124  0.239601  0.041312  0.052664
8    0.225466  0.034713   0.054030  0.220872  0.033538  0.056019
9    0.225126  0.034629   0.054030  0.220872  0.033538  0.056019
10   0.220773  0.043817   0.058961  0.216305  0.04173

KeyboardInterrupt: 